# Step 1:
Here we take the sequences used in the experiment and create a table using that data.

YES-1 riboswitch from Figure 2a

NOT-1 riboswitch in Figure 4a

AND-1 riboswitch in Figure 5a

OR-1 riboswitch in Figure 6a

# Step 2:
Typing in RNAfold into the terminal using the sequence of YES-1 as seen below in seqyes1, gives the following output:(((((((((((((((((((((..(((.......))).)))))))).))))).....(((((....))))).))))))))


In [1]:
import sqlite3
import subprocess

In [2]:
conn = sqlite3.connect('riboswitch.db')
c = conn.cursor()
#c.execute('CREATE TABLE riboswitches (riboswitch TEXT, OBS1 TEXT, OBS2 TEXT, red_region TEXT);')
c.executemany('INSERT INTO riboswitches(riboswitch, OBS1, OBS2, red_region) VALUES(?, ?, ?, ?)',
         [('YES-1', '26-47', '', '16-21, 49-54'),
          ('NOT-1', '45-66', '', '40-44, 74-78'),
          ('AND-1', '30-45', '49-65', '16-23, 70-77'),
         ('OR-1', '27-46', '47-66', '16-26, 67-77')])  
conn.commit()
seqyes1 = 'GGGCGACCCUGAUGAGCUUGGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC'
seqnot1 = 'GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUUGCGUCCUGUAUUCCACUGC'
seqand1 = 'GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU'
seqor1 = 'GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUGCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC'

# Step 3: 
Now, we'll create a routine to model RNA folding.
Typing in the RNAplot command into the terminal command line and using the sequence of yes1 gives the following output. 

Start by generating one plot per riboswitch (YES-1, NOT-1, AND-1, and OR-1) with default parameters.


In [3]:
seqs = \
"""
>seqyes1
GGGCGACCCUGAUGAGCUUGGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
>seqnot1
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUUGCGUCCUGUAUUCCACUGC
>seqand1
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
>seqor1
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUGCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
"""

#def RNAfold (seq):
p = subprocess.run('RNAfold', input = bytes(seqs, 'ascii'), stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
print(p.stderr.decode())
print(p.stdout.decode())
#RNAfold(seqyes1)
#RNAfold(seqnot1)
#RNAfold(seqand1)
#RNAfold(seqor1)




>seqyes1
GGGCGACCCUGAUGAGCUUGGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
(((((((((((((((((((((..(((.......))).)))))))).))))).....(((((....))))).)))))))) (-35.90)
>seqnot1
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUUGCGUCCUGUAUUCCACUGC
.((((....((((((.......((((......))))...((((((.((((..(((......)))..))))..)))))).))))))....)))) (-26.30)
>seqand1
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
((((((((((((((((((((...(((.....(((.(((.......))).))).....)))..))))))).))))).....(((((....))))).))))))))......... (-42.10)
>seqor1
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUGCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((((((((((((((((.(((.....(((.((.((....)))).))).)))...)).))))))).))))).....(((((....))))).)))))))) (-38.00)



## Step 4

Next, simulate OBS binding in YES-1 and NOT-1. 

Use a dot (“.”) for every unconstrained base, and a lower-case x for every base that you want to prevent from pairing. That is to say, put an x at every base spanning the blue OBS region.

In [4]:
# create constrain for yes-1
ofile = open("yes_constrain.txt", "w")
seq = "AAA"
yesCst = "..............xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx................................."
ofile.write(seq+'\n')
ofile.write(yesCst)
ofile.close()

In [44]:
sql = "SELECT riboswitch, OBS1, OBS2\
        FROM riboswitches\
        "
c.execute(sql)
seqObsList = c.fetchall()

In [45]:
def makeConstraintFile(seq, tp):
    ofile = open(tp[0]+".txt", "w")
    # get start and end for OBS1
    obs1 = tp[1].split('-')
    constraint = '.'*(int(obs1[0])-1) + 'x'*(int(obs1[1])-int(obs1[0])+1)
    # get start and end for OBS2
    if len(tp[2]) != 0:
        obs2 = tp[2].split('-')
        constraint = constraint + 'x'*(int(obs2[1])-int(obs2[0])+1) + '.'*(len(seq)-int(obs2[1]))
    ofile.write(seq+'\n')
    ofile.write(constraint)
    ofile.close()
    print(seq)
    print(constraint)
    
makeConstraintFile(seqyes1, c.fetchone())

TypeError: 'NoneType' object is not subscriptable